In [ ]:
%run ../commons/imports_cleaner.ipynb
%run ../commons/helper_db.ipynb

In [ ]:
create_database('blp_dataset')

In [ ]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS image_hashes ("
    "  file TEXT NOT NULL,"
    "  hash_file VARCHAR(255) PRIMARY KEY NOT NULL,"
    "  market VARCHAR(45) NOT NULL DEFAULT 'no'"
    ") ENGINE=InnoDB")
cursor.execute(table_schema)

cursor.execute("DROP TABLE IF EXISTS repeated_image_hashes")

table_schema = (
    "CREATE TABLE IF NOT EXISTS repeated_image_hashes ("
    "  file VARCHAR(255) NOT NULL,"
    "  hash_file TEXT NOT NULL"
    ") ENGINE=InnoDB")
cursor.execute(table_schema)

In [ ]:
## CLEAR REPEATED IMAGES

# == Files config ==
MAIN_FOLDER = r"D:\images\_myntra"

folder = Path(MAIN_FOLDER)
repeated_files = []


for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")

    try:
        img_hash = imagehash.phash(Image.open(image_path))
        image_name = os.path.splitext(image)[0]
        
        cursor.execute("SELECT COUNT(file) FROM image_hashes WHERE hash_file = %s", (str(img_hash),))
        has_hash = cursor.fetchone()

        if has_hash[0] == 0:
            query = "INSERT INTO image_hashes (file, hash_file, market) VALUES (%s, %s, %s)"
            values = (image_name, str(img_hash), "nucleus")

            cursor.execute(query, values)
            conn.commit()
        else:
            query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
            values = (image_name, str(img_hash))

            cursor.execute(query, values)
            conn.commit()
    except:
        query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
        values = (image_name, image_name)

        cursor.execute(query, values)
        conn.commit()



In [ ]:
# REMOVE IMAGES IN REPEATED TABLE
cursor.execute("SELECT * FROM repeated_image_hashes")
repeated = cursor.fetchall()

MAIN_FOLDER = r"D:\images\_myntra"
folder = Path(MAIN_FOLDER)

for res in repeated:
    image_path = Path(f"{folder}\{res[0]}.jpeg")
    
    try:
        os.remove(image_path)
    except:
        print(image_path)

In [ ]:
## CLEAR REPEATED IMAGES IN DIFFERENT FOLDERS

# == Files config ==
MAIN_FOLDER = r"D:\images\_haven"

folder = Path(MAIN_FOLDER)
repeated_files = []


for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")


    img_hash = imagehash.phash(Image.open(image_path))
    image_name = os.path.splitext(image)[0]

    cursor.execute("SELECT COUNT(file) FROM image_hashes WHERE hash_file = %s", (str(img_hash),))
    has_hash = cursor.fetchone()
    
    if has_hash[0] == 0:
        print("Safe")
    else:
        query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
        values = (image_name, str(img_hash))

        cursor.execute(query, values)
        conn.commit()

In [ ]:
# Get list of numbers in sequence that don't have files

count = 0
nums = []
MAIN_FOLDER = r"D:\images\illegal"

while (len(nums) < 174 or count < 58771):
    img = Path(f"{MAIN_FOLDER}\{count}.jpeg")
    if not img.exists():
        nums.append(count)

    count = count + 1


In [ ]:
# Rename images based on list of numbers that don't exist into illegal folder

folder = Path("D:\images\_temp")
folder_tmp = Path("D:\images\_folder_tmp")
cont = 1

for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")

    try:
        img = Image.open(image_path).convert('RGB').save(f'{folder_tmp}\{nums[cont]}.jpeg')
        cont = cont + 1
    except:
        print(image_name)

In [ ]:
print(nums)